#### Imports

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

#### Load question and answer dataset

In [ ]:
dataset = "New dataset\\incident_report_q&a.csv"
df = pd.read_csv(dataset, encoding='latin1') # if csv file in different directory add with correct path

#### Explore nature of dataset

In [ ]:
# Print first 5 rows  
print(df.head(),"\n")
#see last 5 rows 
print(df.tail(),"\n")

print(df.shape) #see number of rows and columns
num_rows = df.shape[0]
print("No of Rows: " + str(df.shape[0])+"\n")
num_columns = df.shape[1]
print("Number of columns in the dataset:", num_columns,"\n")

                      Summary                         Resolution
0    asycuda issue 6 one user    cleared thinapp browder history
1     email service available       outlookemail profile created
2     oracle issue 7 one user  refreshed session cleared browser
3  pc related issues  airport                       payment done
4                 btp working         printer emulator restarted 

                                    Summary  \
98047           network printer working cak   
98048  printer issue airport  laser network   
98049                         email working   
98050                       linemax working   
98051                          sara working   

                                              Resolution  
98047       printer ip checked fixed static ip 102363692  
98048                                    currently issue  
98049  cdmc onprem shared mailbox sn11421 delegated l...  
98050             linemax account extended till 10012026  
98051  per user issue occured 

In [ ]:
# Print column names 
print(df.columns,"\n")

# Print info on data types, non-nulls, memory usage etc.
print(df.info(),"\n")  

# Display summary statistics  
# Set display to show all rows and columns  
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Index(['Summary', 'Resolution'], dtype='object') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98052 entries, 0 to 98051
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Summary     98052 non-null  object
 1   Resolution  98046 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB
None 



In [ ]:
# Print full summary statistics description
print(df.describe()) 

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())   

# Check duplicates
print("\nNumber of duplicate rows:")   
print(df.duplicated().sum())

            Summary                    Resolution
count         98052                         98046
unique         1554                         48624
top     cloud issue  outlookemail profile created
freq           8816                          1246

Missing values per column:
Summary       0
Resolution    6
dtype: int64

Number of duplicate rows:
36790


#### Check for unique values

In [6]:
# Loop through each column in the DataFrame
for column in df.columns:
    unique_values = df[column].unique()
    unique_count = df[column].nunique()
    
    print(f"Column: {column}")
    print(f"Total Unique Values: {unique_count}")
    print(f"Unique Values: {unique_values}")
    print("-" * 40)


    # Print all unique values or limit to first N values if too many
    if unique_count > 20:  # You can change this limit based on your needs
        print(unique_values[:20])  # Print first 20 unique values only
        print("... (and more)")
    else:
        print(unique_values)
    
    print("-" * 40)

Column: Summary
Total Unique Values: 1554
Unique Values: ['asycuda issue 6 one user' 'email service available'
 'oracle issue 7 one user' ... 'staff travel link mentioning display'
 'hit desktop issue' 'agents information portal working']
----------------------------------------
['asycuda issue 6 one user' 'email service available'
 'oracle issue 7 one user' 'pc related issues  airport' 'btp working'
 'charika issue 6 one user' 'application working'
 '2d barcode reader working' 'cloud issue desktop resource available'
 'cute keyboard working airport' 'cloud issue'
 'sabre crew management 6 minor functional users'
 'document scanner issue cak' 'telephone working properly'
 'bcps password reset' 'bpp working' 'skychain issue 7 one user'
 'printer issue airport  laser network' 'document printer working'
 'login related incident']
... (and more)
----------------------------------------
Column: Resolution
Total Unique Values: 48624
Unique Values: ['cleared thinapp browder history' 'outlooke

Downloading NLTK package

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
# Download stopwords from NLTK
nltk.download('stopwords')

### Handling missing values

 Convert to lowercase

In [ ]:
# Fill NaN values in 'Summary' and 'Resolution' columns with empty strings
df['Summary'].fillna('', inplace=True)
df['Resolution'].fillna('', inplace=True)
# Convert 'Summary' and 'Resolution' columns to lowercase
df['Summary_lower'] = df['Summary'].str.lower()
df['Resolution_lower'] = df['Resolution'].str.lower()

Handle punctuation and stopwords

In [ ]:
# Define a function to remove punctuation and stopwords
def preprocess_text(text):
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    # Tokenize the text into words
    words = text.split()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Join words back into a single string
    return ' '.join(words)

In [ ]:
# Create new columns with preprocessed data
df['Preprocessed_Summary'] = df['Summary_lower'].apply(preprocess_text)
df['Preprocessed_Resolution'] = df['Resolution_lower'].apply(preprocess_text)

In [ ]:

# Display the first few rows of the dataframe with new columns
print(df[['Summary', 'Preprocessed_Summary', 'Resolution', 'Preprocessed_Resolution']].head())

                      Summary       Preprocessed_Summary  \
0    asycuda issue 6 one user   asycuda issue 6 one user   
1     email service available    email service available   
2     oracle issue 7 one user    oracle issue 7 one user   
3  pc related issues  airport  pc related issues airport   
4                 btp working                btp working   

                          Resolution            Preprocessed_Resolution  
0    cleared thinapp browder history    cleared thinapp browder history  
1       outlookemail profile created       outlookemail profile created  
2  refreshed session cleared browser  refreshed session cleared browser  
3                       payment done                       payment done  
4         printer emulator restarted         printer emulator restarted  


In [ ]:
# Save the modified dataframe with new columns to a CSV file (if needed)
#df.to_csv("D:\\User Data\\Oshadi\\USJ\\Acedemic\\3rd Year\\Sem 6\\Project\\Test1\\clean_and_encoding\\dataset\\question_and_answers_preprocessed.csv", index=False)
df.to_csv("New dataset\\incident_report_q&a_2.csv")


 Group by 'Summary' and find the most common resolution and fill missing values in 'Resolution'

In [ ]:
import numpy as np


dataset = "New dataset\\incident_report_q&a_2.csv"
df = pd.read_csv(dataset, encoding='latin1') # if csv file in different directory add with correct path

# Print original DataFrame (for debugging)
print("Original DataFrame:")
print(df.head())

# Fill missing values in 'Resolution' based on the most common resolution for each summary
# Step 1: Group by 'Summary' and find the most common resolution
resolution_mode = df.groupby('Summary')['Resolution'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

# Step 2: Map the most common resolution back to the original DataFrame
df['Resolution'] = df.apply(lambda row: resolution_mode[row['Summary']] if pd.isna(row['Resolution']) else row['Resolution'], axis=1)

# Optionally, you can also fill any remaining missing values with a specific value
# df['Resolution'].fillna('missing_resolution', inplace=True)

# Print modified DataFrame
print("\nModified DataFrame:")
print(df.head())


Original DataFrame:
   Unnamed: 0                     Summary                         Resolution  \
0           0    asycuda issue 6 one user    cleared thinapp browder history   
1           1     email service available       outlookemail profile created   
2           2     oracle issue 7 one user  refreshed session cleared browser   
3           3  pc related issues  airport                       payment done   
4           4                 btp working         printer emulator restarted   

                Summary_lower                   Resolution_lower  \
0    asycuda issue 6 one user    cleared thinapp browder history   
1     email service available       outlookemail profile created   
2     oracle issue 7 one user  refreshed session cleared browser   
3  pc related issues  airport                       payment done   
4                 btp working         printer emulator restarted   

        Preprocessed_Summary            Preprocessed_Resolution  
0   asycuda issue 6 one 

#### Save to new CSV (no missing values)

In [ ]:
df.to_csv("New dataset\\incident_report_q&a_2_no_missings.csv", index=False)

#### Group by summery

 Get the Count of Summary Groups and Count and Print Different Summary Groups

In [ ]:
import numpy as np
import pandas as pd

dataset = "New dataset\\incident_report_q&a_2_no_missings.csv"
df = pd.read_csv(dataset, encoding='latin1')  # Load CSV with latin1 encoding

# Group by 'Summary' and count the number of occurrences of each group
summary_counts = df.groupby('Summary').size()

# Print the summary groups and their counts
print("\nSummary Groups and Counts:")
print(summary_counts)

# Get the total number of unique 'Summary' groups
unique_summary_count = df['Summary'].nunique()

# Print the total count of unique 'Summary' groups
print(f"\nTotal number of unique Summary groups: {unique_summary_count}")

# Calculate the mode for the 'Resolution' within each 'Summary' group
resolution_mode = df.groupby('Summary')['Resolution'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

# Fill missing 'Resolution' values with the mode from each 'Summary' group
df['Resolution'] = df.apply(lambda row: resolution_mode[row['Summary']] if pd.isna(row['Resolution']) else row['Resolution'], axis=1)





Summary Groups and Counts:
Summary
12641 visible gal                                                                             1
17279 charika updates                                                                         1
2031153595 location airport counter 161                                                       1
2031153795 location airport counter 161                                                       1
2d barcode reader working                                                                   422
43 ul theme ip phones                                                                         1
abi analyzer download error                                                                   1
able connect ul vpn                                                                           1
acars application issue                                                                       1
acars issue                                                                                  10
acar

Validate the Sum of Group Counts

In [18]:
# Sum of all counts in 'Summary' groups
total_group_counts = summary_counts.sum()

# Total number of rows in the dataset
total_rows = len(df)

# Validate that the sum of group counts equals the total number of rows
if total_group_counts == total_rows:
    print(f"\nSum of counts in each group ({total_group_counts}) matches the total number of rows ({total_rows}).")
else:
    print(f"\nMismatch: Sum of counts in each group ({total_group_counts}) does not match the total number of rows ({total_rows}).")



Sum of counts in each group (98052) matches the total number of rows (98052).


#### Create new question and answer dataset 

In [ ]:
import pandas as pd

# Select the required columns and rename them
df_new = df[['Preprocessed_Summary', 'Preprocessed_Resolution']].rename(
    columns={'Preprocessed_Summary': 'Summary', 'Preprocessed_Resolution': 'Resolution'}
)

# Save the new dataset (optional)
df_new.to_csv("New dataset\\incident_report_q&a", index=False)

# Display the first few rows
print(df_new.head())


#### Explore new question and answer dataset

In [15]:
import pandas as pd
dataset="d:\\User Data\\Oshadi\\USJ\\Acedemic\\3rd Year\\Sem 6\\Project\\Test1\\clean_and_encoding\\dataset\\New dataset\\incident_report_q&a.csv"
df = pd.read_csv(dataset, encoding='latin1') 


print(df.shape) #see number of rows and columns
num_rows = df.shape[0]
print("No of Rows: " + str(df.shape[0])+"\n")
num_columns = df.shape[1]
print("Number of columns in the dataset:", num_columns,"\n")

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())   



(98052, 2)
No of Rows: 98052

Number of columns in the dataset: 2 


Missing values per column:
Summary       0
Resolution    6
dtype: int64


#### Check duplicates in new dataset

In [16]:
# Check duplicates
print("\nNumber of duplicate rows:")   
print(df.duplicated().sum())


Number of duplicate rows:
36790


#### Remove duplicates and handle if missing values occured

In [3]:
import pandas as pd

# Load the dataset
dataset="d:\\User Data\\Oshadi\\USJ\\Acedemic\\3rd Year\\Sem 6\\Project\\Test1\\clean_and_encoding\\dataset\\New dataset\\incident_report_q&a.csv"
df = pd.read_csv(dataset, encoding='latin1') 
# Display initial information about the dataset
initial_info = {
    "initial_shape": df.shape,
    "columns": df.columns.tolist(),
    "missing_values": df.isnull().sum(),
    "duplicates_count": df.duplicated().sum()
}
# 1. Remove duplicate rows
df = df.drop_duplicates()
# 2. Handle missing values by dropping rows with missing data
df = df.dropna()
# 3. Text cleaning: convert text to lowercase, remove punctuation, and strip whitespace
df['Summary'] = df['Summary'].str.lower().str.replace(r'[^\w\s]', '', regex=True).str.strip()
df['Resolution'] = df['Resolution'].str.lower().str.replace(r'[^\w\s]', '', regex=True).str.strip()
# Display final shape and sample data after preprocessing
final_info = {
    "final_shape": df.shape,
    "sample_data": df.head()
}

initial_info, final_info


({'initial_shape': (98052, 2),
  'columns': ['Summary', 'Resolution'],
  'missing_values': Summary       0
  Resolution    6
  dtype: int64,
  'duplicates_count': 36790},
 {'final_shape': (61256, 2),
  'sample_data':                       Summary                         Resolution
  0    asycuda issue 6 one user    cleared thinapp browder history
  1     email service available       outlookemail profile created
  2     oracle issue 7 one user  refreshed session cleared browser
  3  pc related issues  airport                       payment done
  4                 btp working         printer emulator restarted})

#### Final Dataset for Question and Answering

In [ ]:
# Save the cleaned dataset to a new CSV file
output_path = 'd:\\User Data\\Oshadi\\USJ\\Acedemic\\3rd Year\\Sem 6\\Project\\Test1\\clean_and_encoding\\dataset\\New dataset\\incident_report_q&a_cleaned.csv'
df.to_csv(output_path, index=False)

print("File saved as:", output_path)

#### Explore Final Full Question and Answering

In [4]:
import pandas as pd
dataset="d:\\User Data\\Oshadi\\USJ\\Acedemic\\3rd Year\\Sem 6\\Project\\Test1\\clean_and_encoding\\dataset\\New dataset\\incident_report_q&a_cleaned.csv"
df = pd.read_csv(dataset, encoding='latin1') 


print(df.shape) #see number of rows and columns
num_rows = df.shape[0]
print("No of Rows: " + str(df.shape[0])+"\n")
num_columns = df.shape[1]
print("Number of columns in the dataset:", num_columns,"\n")

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())   

# Check duplicates
print("\nNumber of duplicate rows:")   
print(df.duplicated().sum())

(61256, 2)
No of Rows: 61256

Number of columns in the dataset: 2 


Missing values per column:
Summary       0
Resolution    0
dtype: int64

Number of duplicate rows:
0
